# HW 2 Project Part II
#### <u>Group 8: Annaston Evers, Juliette Vasquez, Madison Gaines, Mrityunjay Sivakumar, Shirley Lin, Uday Thakar, Victor Irby<u>
##### *An <ins>IF model</ins> is also known as an **integrate-and-fire model** that represents a neuron as an electrical circuit.*
##### **It depicts how the circuit receives incoming signals over time until a threshold is reached to fire and reset. This is to simulate the production of an action potential in neurons.**
[Link to More Info](https://neuronaldynamics.epfl.ch/online/Ch1.S3.html)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
V_rest = -65  # Resting potential (mV)
V_th = -55    # Threshold potential (mV)
V_reset = V_rest  # Reset potential after firing (mV)
tau = 20  # Membrane time constant (ms)
R = 10  # Membrane resistance (MΩ)
dt = 0.1  # Time step (ms)
T = 100  # Total time for the simulation (ms)

# Function to simulate the integrate-and-fire model
def integrate_and_fire(I_ext, T=100, dt=0.1):
    # Initialize time and membrane potential arrays
    t = np.arange(0, T, dt)
    V = np.full_like(t, V_rest)  # Membrane potential (initially resting potential)

    # Iterate over each time step
    for i in range(1, len(t)):
        # Differential equation for membrane potential
        dV = (- (V[i-1] - V_rest) + R * I_ext) * dt / tau
        V[i] = V[i-1] + dV

        # Check if the potential exceeds the threshold
        if V[i] >= V_th:
            V[i] = V_reset  # Reset after firing

    return t, V

# Simulate with different input currents (e.g., 2 levels of input current)
I_ext_low = 1.5  # Low input current (nA)
I_ext_high = 3.0  # High input current (nA)

# Get results for both currents
t_low, V_low = integrate_and_fire(I_ext_low)
t_high, V_high = integrate_and_fire(I_ext_high)

# Plotting the results
plt.figure(figsize=(10, 6))

# Low current input
plt.subplot(2, 1, 1)
plt.plot(t_low, V_low, label=f'I_ext = {I_ext_low} nA')
plt.axhline(y=V_th, color='r', linestyle='--', label="Threshold (-55 mV)")
plt.axhline(y=V_rest, color='g', linestyle='--', label="Resting Potential (-65 mV)")
plt.title('Firing Pattern with Low Input Current')
plt.xlabel('Time (ms)')
plt.ylabel('Membrane Potential (mV)')
plt.legend(loc='upper right')

# High current input
plt.subplot(2, 1, 2)
plt.plot(t_high, V_high, label=f'I_ext = {I_ext_high} nA', color='orange')
plt.axhline(y=V_th, color='r', linestyle='--', label="Threshold (-55 mV)")
plt.axhline(y=V_rest, color='g', linestyle='--', label="Resting Potential (-65 mV)")
plt.title('Firing Pattern with High Input Current')
plt.xlabel('Time(ms)')
plt.ylabel('Membrane Potential(mV)')
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()


### Visualization of Results
The plots below illustrate the firing patterns of a neuron under low and high input current conditions.
![IF Model Results](graph.jpg)


### Question 3
**Looking at the above visualization / based on what you know regarding the LIF model -- what aspect of this model is most unlike a real biological neuron? What is missing? Additionally what aspects are like a biological neuron? Compare and Contrast the two.**

In terms of aspects missing in this graph, our model is missing a refractory period, action potentials, and leaky channels when compared to a real biological neuron. This model is similar to real neurons in the way that the response to input currents is non-linear voltage changes. Furthermore, there is also a resting membrane potential and appropriate membrane threshold that the neuron reaches to before dropping back to resting.

### Question 4
**What was the most challenging aspect of this homework?**

The most challenging aspect of this homework was navigating GitHub, especially importing our image results.

### Question 5
**Estimate how long it took you to complete this homework**

Approximately 1.5 hours.

# HW 3 Project Part III
**The <ins>LIF model</ins> builds upon the IF model to introduce leaky ion channels, which allows for a more natural decay in voltage upon reaching threshold, rather than the sharp reset seen with the IF model.**

**The <ins>HH model</ins> is an even more accurate model of neuron firing, incorporating the voltage-gated sodium and potassium channels instead of a simple spike generator.**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Hodgkin-Huxley Model Parameters
V_rest = -65  # Resting potential (mV)
C_m = 1.0  # Membrane capacitance (uF/cm^2)
E_Na = 50  # Sodium reversal potential (mV)
E_K = -77  # Potassium reversal potential (mV)
E_L = -54.4  # Leak reversal potential (mV)
G_Na = 120  # Sodium conductance (mS/cm^2)
G_K = 36  # Potassium conductance (mS/cm^2)
G_L = 0.3  # Leak conductance (mS/cm^2)

# Gating Variables (HH Model Activation & Inactivation Functions)
def alpha_n(V): return 0.01 * (V + 55) / (1 - np.exp(-(V + 55) / 10))
def beta_n(V): return 0.125 * np.exp(-(V + 65) / 80)
def alpha_m(V): return 0.1 * (V + 40) / (1 - np.exp(-(V + 40) / 10))
def beta_m(V): return 4 * np.exp(-(V + 65) / 18)
def alpha_h(V): return 0.07 * np.exp(-(V + 65) / 20)
def beta_h(V): return 1 / (1 + np.exp(-(V + 35) / 10))

# Hodgkin-Huxley Model Function
def hh_model(I_ext, T=250, dt=0.01):
    t = np.arange(0, T, dt)
    V = np.full_like(t, V_rest)
    n = 0.3177  # Initial value from steady state
    m = 0.0529
    h = 0.5961
    I = np.zeros_like(t)
    I[5000:15000] = I_ext  # Apply stimulus between 50ms and 150ms
    
    for i in range(1, len(t)):
        # Update gating variables
        dn = (alpha_n(V[i-1]) * (1 - n) - beta_n(V[i-1]) * n) * dt
        dm = (alpha_m(V[i-1]) * (1 - m) - beta_m(V[i-1]) * m) * dt
        dh = (alpha_h(V[i-1]) * (1 - h) - beta_h(V[i-1]) * h) * dt
        
        n += dn
        m += dm
        h += dh
        
        # Compute ionic currents
        I_Na = G_Na * m**3 * h * (V[i-1] - E_Na)
        I_K = G_K * n**4 * (V[i-1] - E_K)
        I_L = G_L * (V[i-1] - E_L)
        
        # Membrane voltage update
        dV = (I[i] - (I_Na + I_K + I_L)) / C_m * dt
        V[i] = V[i-1] + dV
    
    return t, V, I

# Simulating HH model with different input currents
I_ext1 = 5.0  # Lower external current (not enough to trigger multiple spikes)
I_ext2 = 15.0  # Stronger burst input

t_hh1, V_hh1, I_hh1 = hh_model(I_ext1)
t_hh2, V_hh2, I_hh2 = hh_model(I_ext2)

# Plot Results
plt.figure(figsize=(10, 6))

# HH Model Membrane Potential for different inputs
plt.subplot(2, 1, 1)
plt.plot(t_hh1, V_hh1, label='Lower Input (5 uA/cm^2)', color='blue')
plt.plot(t_hh2, V_hh2, label='Burst Input (15 uA/cm^2)', color='orange')
plt.axhline(y=-55, color='r', linestyle='--', label="Threshold (-55 mV)")
plt.axhline(y=V_rest, color='g', linestyle='--', label="Resting Potential (-65 mV)")
plt.title('Hodgkin-Huxley Model with Different Inputs')
plt.xlabel('Simulation Time (milliseconds)')
plt.ylabel('Membrane Potential (mV)')
plt.legend()

# Stimulus Current Plot
plt.subplot(2, 1, 2)
plt.plot(t_hh1, I_hh1, label='Lower Input', color='blue', linestyle='--')
plt.plot(t_hh2, I_hh2, label='Burst Input', color='orange', linestyle='--')
plt.xlabel('Simulation Time (milliseconds)')
plt.ylabel('Stimulus (uA/cm^2)')
plt.legend()

plt.tight_layout()
plt.show()

# Explanation of Changes
print("\nExplanation:")
print("- The input current was modified from 10 uA/cm^2 to 5 uA/cm^2 (lower input) and 15 uA/cm^2 (burst input).")
print("- The lower input did not reach the threshold enough times to trigger repeated spikes, leading to fewer action potentials.")
print("- The burst input provided a stronger stimulus, causing more frequent and sustained spikes, mimicking real neuronal burst firing.")


### Visualization of Differing Inputs
The plots below illustrate the firing patterns if the inputs given differ.
![HH Model Results](Screen Shot 2025-02-03 at 6.42.32 PM.png)


### Discussion Questions
**What changes had to be made to the model and why?**

answer here.

**How did you and/or ChatGPT implement the changes?**

answer here.

**Define 'bits' and Shannon's entropy with relation to information theory. Which model contains the most bits? Which model has the highest Shannon's entropy? Explain your answer.**

Bits is a unit of measure that predicts uncertainty and it quantifies the amount of information where 1 bit is equivalent to a unit of information. In relation to information theory a bit quantifies the information that is stored. Shannon's entropy is the amount of uncertainty that is involved when generating random variables. In relation to information theory, this describes the amount of uncertainty or unpredictability in neural processes to compare information amounts. The HH model has the most bits because it has more variables with its ion channels which uses more bits. The HH model has the highest Shannon's entropy because there are more variables and more unpredictable behavior.